In [1]:
import tensorflow as tf
import os
import zipfile

DESIRED_ACCURACY = 0.999

!wget --no-check-certificate \
    "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip" \
    -O "/tmp/happy-or-sad.zip"

zip_ref = zipfile.ZipFile("/tmp/happy-or-sad.zip", 'r')
zip_ref.extractall("/tmp/h-or-s")
zip_ref.close()

class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')>DESIRED_ACCURACY):
      print("\nReached 99.9% accuracy so cancelling training!")
      self.model.stop_training = True

callbacks = myCallback()


--2020-06-03 05:27:21--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.199.128, 2607:f8b0:400e:c09::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.199.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2670333 (2.5M) [application/zip]
Saving to: ‘/tmp/happy-or-sad.zip’

/tmp/happy-or-sad.z 100%[===================>]   2.55M  --.-KB/s    in 0.01s   

2020-06-03 05:27:21 (171 MB/s) - ‘/tmp/happy-or-sad.zip’ saved [2670333/2670333]



In [0]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

from tensorflow.keras.optimizers import RMSprop

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=0.001),
              metrics=['accuracy'])

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1/255)

train_generator = train_datagen.flow_from_directory(
        "/tmp/h-or-s",  
        target_size=(150, 150), 
        batch_size=10,
        class_mode='binary')


Found 80 images belonging to 2 classes.


In [4]:
history = model.fit(
      train_generator,
      steps_per_epoch=8,  
      epochs=15,
      verbose=1,
      callbacks=[callbacks])

Epoch 1/15
8/8 [==============================] - 2s 195ms/step - loss: 1.5479 - accuracy: 0.6750
Epoch 2/15
8/8 [==============================] - 2s 194ms/step - loss: 0.5805 - accuracy: 0.7500
Epoch 3/15
8/8 [==============================] - 2s 192ms/step - loss: 0.3504 - accuracy: 0.8250
Epoch 4/15
8/8 [==============================] - 2s 191ms/step - loss: 0.1483 - accuracy: 0.9750
Epoch 5/15
8/8 [==============================] - 2s 190ms/step - loss: 0.1335 - accuracy: 0.9375
Epoch 6/15
8/8 [==============================] - 2s 190ms/step - loss: 0.1392 - accuracy: 0.9500
Epoch 7/15
8/8 [==============================] - 2s 191ms/step - loss: 0.0781 - accuracy: 0.9625
Epoch 8/15
8/8 [==============================] - 2s 190ms/step - loss: 0.0571 - accuracy: 0.9750
Epoch 9/15
8/8 [==============================] - 2s 189ms/step - loss: 0.1539 - accuracy: 0.9000
Epoch 10/15
8/8 [==============================] - 2s 194ms/step - loss: 0.0473 - accuracy: 0.9875
Epoch 11/15
8/8 [==